In [2]:
"""
设置
    1、随机请求头
    2、随机IP
试错机制
    1、出错继续执行下一条链接
    2、并将该出错的链接存入列表中，留后续执行完所有链接后从新执行
    3、出错后返回
        （1）出错时的链接
        （2）出错时的IP代理
        （3）出错时的请求头
        （4）出错时的错误 exception 
"""
import random
import requests
import pymongo
from lxml import etree
from parsel import Selector
client = pymongo.MongoClient(host='localhost', port=27017)
db     = client['indianhospital']
col    = db['hospital']
error_province_col = db['error_province']
error_hospital_col = db['error_hospital']

headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'
}

"""
一、获取 37 个省份的链接
返回一个包含 37 个省份链接的列表
"""
first_layer_url = 'https://www.medindia.net/patients/hospital_search/hospital_list.asp?utm_source=topnavigation&utm_medium=desktop&utm_content=&utm_campaign=medindia'
proxies = {'http' : 'http://' + requests.get('http://127.0.0.1:5555/random').text.strip()}
try:
    first_layer_res = requests.get(
        url     = first_layer_url,
        headers = headers,
        proxies = proxies
    )
    html = etree.HTML(first_layer_res.text)
    province_href_list = html.xpath('//ul[@class="list-inline"]/div/div/li/a/@href')
    print(province_href_list)  
except:
    print('【ERROR】')
    print('\t获取省份失败')

['https://www.medindia.net/patients/hospital_search/hospitals-andaman-and-nicobar-islands.htm', 'https://www.medindia.net/patients/hospital_search/hospitals-andhra-pradesh.htm', 'https://www.medindia.net/patients/hospital_search/hospitals-arunachal-pradesh.htm', 'https://www.medindia.net/patients/hospital_search/hospitals-assam.htm', 'https://www.medindia.net/patients/hospital_search/hospitals-bihar.htm', 'https://www.medindia.net/patients/hospital_search/hospitals-chandigarh.htm', 'https://www.medindia.net/patients/hospital_search/hospitals-chhattisgarh.htm', 'https://www.medindia.net/patients/hospital_search/hospitals-dadra-and-nagar-haveli.htm', 'https://www.medindia.net/patients/hospital_search/hospitals-daman-and-diu.htm', 'https://www.medindia.net/patients/hospital_search/hospitals-delhi.htm', 'https://www.medindia.net/patients/hospital_search/hospitals-goa.htm', 'https://www.medindia.net/patients/hospital_search/hospitals-gujarat.htm', 'https://www.medindia.net/patients/hospital

In [3]:
"""
二、遍历上面的 37 个省份的链接（通过 上面的列表），
返回每个省份包含的所有医院链接（暂时不设置翻页）的列表
"""
for province in province_href_list:
    proxies = {'http' : 'http://' + requests.get('http://127.0.0.1:5555/random').text.strip()}
    try:
        res = requests.get(
            url = province,
            headers=headers,
            proxies = proxies
        )
        html = etree.HTML(res.text)
        hospital_href_list = html.xpath('//h3[@class="vert-small-margin"]/a/@href')
        print('【SUCCDDE hospital_href_list】')

        """测试 hospital_href_list"""
        for hospital in hospital_href_list:
            proxies = {'http' : 'http://' + requests.get('http://127.0.0.1:5555/random').text.strip()}
            try:
                res = requests.get(
                    url=hospital,
                    headers=headers,
                    proxies=proxies
                )
                print('【SUCCEED hospital】')
                selector = Selector(text = res.text)
                hospital_name = selector.xpath('//div[@class="mi-bg-1"]/../h2/text()').re_first('Address of (.*)')
                address       = ', '.join(selector.xpath('//div[@class="mi-bg-1"]/div/div/div[contains(@class, "report-content")]/p[1]//text()').re('\s*(\w.*\w)\s*,*'))
                director      = selector.xpath('//div[@class="mi-bg-1"]/div/div/div[contains(@class, "report-content")]/p/b[contains(text(), "Director")]/../text()').re_first('\s*(\w.*\w)\s*')
                email         = selector.xpath('//div[@class="mi-bg-1"]/div/div/div[contains(@class, "report-content")]/p/b[contains(text(), "Email")]/../span/text()').re_first('\s*(\w.*\w)\s*')
                phone         = selector.xpath('//div[@class="mi-bg-1"]/div/div/div[contains(@class, "report-content")]/p/b[contains(text(), "Phone")]/../span/text()').re_first('\s*(\w.*\w)\s*')
                mobile        = selector.xpath('//div[@class="mi-bg-1"]/div/div/div[contains(@class, "report-content")]/p/b[contains(text(), "Mobile")]/../span/text()').re_first('\s*(\w.*\s)\s*')
                hospital_info = {
                    'name'    : hospital_name,
                    'address' : address,
                    'director': director,
                    'email'   : email,
                    'phone'   : phone,
                    'mobile'  : mobile,
                    'proxies' : proxies,
                    'status_code': res.status_code,
                    'res_text': res.text
                }
                
                col.insert_one(hospital_info)   # 医院信息入库
            except:
                print('【ERROR hospital】')
                print('\t', hospital)
                print('\t', proxies)
                error_hospital = {
                    'province':province,
                    'hospital':hospital,
                    'proxies' :proxies
                }
                error_hospital_col.insert_one(error_hospital)   # 错误医院信息入库
    except:
        print('【ERROR hospital_href_list】')
        error_province = {
            'province':province,
            'proxies' :proxies
        }
        error_province_col.insert_many(error_province)           # 错误省份入库